# Zero to Hero

## 2.02 Optimizacion Bayesiana DOS variables

### El código

( por favor, verificar **antes** que se tienen instaladas todas las librerias necesarias, en particular plotly )

In [ ]:
# Ejemplo de optimizacion bayesiana, univariada

# limpio la memoria
rm(list = ls()) # remove all objects
gc() # garbage collection

require("data.table")
require("plotly")

# paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")

options(repr.plot.width = 20, repr.plot.height = 20)


### Defino la funcion a optimizar

In [ ]:
func_volcano <- function(x) {
  z <- volcano[x$b, x$a]

  return(z)
}


Grafico el volcan

In [ ]:
p <- plot_ly(z = volcano, type = "surface")
p


La funcion tiene varios maximos locales

* **fn**    es el nombre de la funcion
* **minimize**   por defecto es TRUE,  pero en este caso le asigno FALSE, ya que deseo maximizar  el valor
* **par.set**  indica cuales son los hiperparmetros de la funcion, en este caso hay una sola variable que llamé  x
---
* **makeIntegerParam** indica que ese hiperparámetro es un numero entero ( que NO es un numero real continuo )

In [ ]:
configureMlr(show.learner.output = FALSE)

obj.fun <- makeSingleObjectiveFunction(
   fn = func_volcano,
   minimize = FALSE, # estoy Maximizando la ganancia
   has.simple.signature = FALSE, # porque tengo DOS dimensiones
   par.set = makeParamSet(
      makeIntegerParam("a", lower = 1, upper = 61),
      makeIntegerParam("b", lower = 1, upper = 87)
   ),
)


ahora defino la funcion proxy, la que se construye internamente intentando emular la realidad

* **cl**  es la clase de learner,  "reg.km" indica el método de kriging "**reg**ression **k**riging **m**ethodd"
* **predict.type**  es el tipo de prediccion que deseo me devuelva, "se" significa que espero dos valores  media y standard error
* **covtype**    es la funcion de covarianza que se va a utilizar, cual es la covarianza de dos mediciones como fucion de la distancia entre los puntos donde fueron tomadas las mediciones, fue inventada por Bertil Matérn 

El método de kriging puede verse en su forma original (la geología) , esta presentación https://www.youtube.com/watch?v=ZB7GvJrNKq0

In [ ]:
fproxy <- makeLearner(
    cl = "regr.km",
    predict.type = "se",
    covtype = "matern3_2"
)


ultima definicion,  especificar la optimizacion bayesiana
*  **crit**   indica el criterio con el que se completan los valores iniciales "no inteligentes"
*  **iters**  indica la cantidad de iteraciones inteligentes que hará la Optimizacion Bayesiana, las que son adicionales a las primeras cuatro de este caso.*  **iters**  indica la cantidad de iteraciones inteligentes que hará la Optimizacion Bayesiana, las que son adicionales a las primeras cuatro de este caso.

In [ ]:
ctrl <- makeMBOControl()
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())
ctrl <- setMBOControlTermination(ctrl, iters = 20)


finalmente , lanzo la Optimizacion Bayesiana

* **fun**  es la especificacion de la funcion que deseo optimizar, si maximizo o minimizo, cuales son las variables de la misma
* **learner**  especifica cual es la función proxy interna que va a utilizar la Optimziación Bayesiana
* **control**  indica la la forma en que se harán las iteraciones

In [ ]:
run <- mbo(
    fun = obj.fun,
    learner = fproxy,
    control = ctrl
)


In [ ]:
tb_resultados <- as.data.table(run$opt.path)


In [ ]:
tb_resultados


In [ ]:
tb_resultados[which.max(tb_resultados$y)]


### Conclusiones

* ¿Qué tan lejos estoy de encontrar el máximo absoluto con 25 iteraciones?
* ¿Qué sucede si aumento la cantidad de iteraciones?